# Tokenization

# Packages

datasets function might need to be revised, in case this is part of the script uploaded to cluster
--> depends where the data is coming from, might be able to upload somewhere. If uploaded to HF for example, we can't use load_from_disk, but have to use load_datasets

In [1]:
from datasets import load_from_disk
from transformers import AutoTokenizer
import torch
from trl import SFTConfig, SFTTrainer

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
raw_datasets = load_from_disk("../data/mbti_dict")

# raw_train_dataset = raw_datasets["train"]
# raw_validation_dataset = raw_datasets["validation"]
# raw_test_dataset = raw_datasets["test"]


# for running local test
small_datasets = raw_datasets.filter(lambda _, indices: indices % 100 == 0, with_indices=True)

raw_train_dataset = small_datasets["train"]
raw_validation_dataset = small_datasets["validation"]
raw_test_dataset = small_datasets["test"]

In [4]:
small_datasets
#raw_train_dataset.features



DatasetDict({
    train: Dataset({
        features: ['label', 'post', '__index_level_0__'],
        num_rows: 2965
    })
    test: Dataset({
        features: ['label', 'post', '__index_level_0__'],
        num_rows: 371
    })
    validation: Dataset({
        features: ['label', 'post', '__index_level_0__'],
        num_rows: 371
    })
})

In [5]:
qwen_checkpoint = "Qwen/Qwen2.5-0.5B-Instruct"
qwen_tokenizer = AutoTokenizer.from_pretrained(qwen_checkpoint)


# Chat-Templates and Tokenization

In [6]:
test_sentence = raw_train_dataset["post"][0]
print(test_sentence)

tokenized_sentence = qwen_tokenizer(test_sentence)
print(tokenized_sentence)

print(qwen_tokenizer.convert_ids_to_tokens(tokenized_sentence["input_ids"]))

i'm extremely irritated of most bugs, like when i was last camping i was a total nervous wreck because of all the horseflies, or when a spider appears descending right above me when i'm relaxing in...
{'input_ids': [72, 2776, 9016, 83871, 315, 1429, 22551, 11, 1075, 979, 600, 572, 1537, 32773, 600, 572, 264, 2790, 22596, 35750, 1576, 315, 678, 279, 15223, 53919, 11, 476, 979, 264, 34354, 7952, 43084, 1290, 3403, 752, 979, 600, 2776, 33848, 304, 1112], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['i', "'m", 'Ġextremely', 'Ġirritated', 'Ġof', 'Ġmost', 'Ġbugs', ',', 'Ġlike', 'Ġwhen', 'Ġi', 'Ġwas', 'Ġlast', 'Ġcamping', 'Ġi', 'Ġwas', 'Ġa', 'Ġtotal', 'Ġnervous', 'Ġwreck', 'Ġbecause', 'Ġof', 'Ġall', 'Ġthe', 'Ġhorse', 'flies', ',', 'Ġor', 'Ġwhen', 'Ġa', 'Ġspider', 'Ġappears', 'Ġdescending', 'Ġright', 'Ġabove', 'Ġme', 'Ġwhen', 'Ġi', "'m", 'Ġrelaxing', 'Ġin', '...']


In [7]:

def convert_to_chatml(data):
    
    prompt = f"Your personality Type is {data["label"]}. What is on your mind?"
    return {
        "messages": [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": data["post"]}
        ]
    }

In [ ]:

ds = small_datasets.map(convert_to_chatml)
print(ds["train"][0]["messages"])

messages = ds["train"][0]["messages"]

text = qwen_tokenizer.applychat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True
)

model_inputs = qwen_tokenizer([text]), return_tensors = "pt"

Map:   0%|          | 0/2965 [00:00<?, ? examples/s]

Map:   0%|          | 0/371 [00:00<?, ? examples/s]

Map:   0%|          | 0/371 [00:00<?, ? examples/s]

[{'content': 'Your personality Type is 9. What is on your mind?', 'role': 'user'}, {'content': "i'm extremely irritated of most bugs, like when i was last camping i was a total nervous wreck because of all the horseflies, or when a spider appears descending right above me when i'm relaxing in...", 'role': 'assistant'}]


In [ ]:
training_args = SFTConfig(
    output_dir = "./mbti_test_output",
    max_steps = 100,
    per_device_train_batch_size = 1,
    learning_rate = 5e-5,
    logging_steps = 10,
    save_steps=100,
    eval_strategy = "steps",
    eval_steps = 50,
    fp16=True
)

In [ ]:
trainer = SFTTrainer(
    model=qwen_checkpoint,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    dataset_text_field = "messages",
    processing_class=qwen_tokenizer,
)

In [ ]:
def convert_to_chatml(data):
    
    prompt = f"Your personality Type is {data['label']}. What is on your mind?"
    return {
        "messages": [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": data["post"]}
        ]
    }

ds = small_datasets.map(convert_to_chatml)
# print(ds["train"][0]["messages"])

# messages = ds["train"][0]["messages"]

# text = qwen_tokenizer.apply_chat_template(
#     messages,
#     tokenize = False,
#     add_generation_prompt = True
# )

#model_inputs = qwen_tokenizer([text], return_tensors = "pt")

training_args = SFTConfig(
    output_dir = "./mbti_test_output",
    max_steps = 100,
    per_device_train_batch_size = 1,
    learning_rate = 5e-5,
    logging_steps = 10,
    save_steps=100,
    eval_strategy = "steps",
    eval_steps = 50,
    fp16=True
)

trainer = SFTTrainer(
    model=qwen_checkpoint,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    dataset_text_field = "messages",
    processing_class=qwen_tokenizer,
)

trainer.train()